In [2]:
import torch

### Comparison

#### CNN
+ 权重共享 平移不变性、可并行计算
+ 滑动窗口 局部关联性建模、依靠多层堆积来进行长程建模
+ 对相对位置敏感、对绝对位置不敏感

#### RNN 依次有序递归建模
+ 对顺序敏感
+ 串行计算耗时
+ 长程建模能力弱
+ 计算复杂度与序列长度呈线性关系
+ 单步计算复杂度不变
+ 对相对位置敏感，对绝对位置敏感
  
#### tranformer
+ 无局部假设
  + 可并行计算
  + 对相对位置不敏感
+ 无有序假设
  + 需要位置编码反映位置变化对特征的影响
  + 对绝对位置不敏感
+ 任意两字符都可以建模
  + 擅长长短程建模
  + 自注意力机制需要序列长度的平方级复杂运算

### Transformer encoder
+ input word embedding
  由稀疏的one-hot进入一个不带bias的FFN得到一个稠密的连续箱梁
+ position encoding
  + 通过sin/cos来固定表征
    + 每个位置确定
    + 对于不同的句子，相同位置的距离一致
    + 可以推广到更长的测试句子
  + pe(pos+k)可以写成pe(k)的线性组合
  + 通过残差连接来是的位置信息流入深层
+ multi-head self-attention
  + 建模能力更强，表征空间更丰富
  + 由多组Q\K\V构成，每组单独计算一个attention向量
  + 把每组的attention向量拼起来，并进入一个FFN中得到最终的向量
+ feed-forward network
  + 只考虑每个单独位置进行建模
  + 不同位置参数共享
  + 类似于1*1 pointwise convolution

### Transformer Decoder
+ output word embedding
+ masked multi-head self-attention
+ multi-head cross-attention
+ feed-forward network
+ softmax

### 自回归模型
+ word embedding
+ position embeding
+ encoder self-attention mask
+ intra-attention mask
+ decoder self-attention mask
+ multi-head self-attention

### 1. work embedding

In [3]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

# word embedding, 以序列建模为例
# 考虑source sentence target sentence
# 这里考虑离散的建模
# 构建序列，序列字符以其在词表索引形式表示

batch_size = 2
src_len = torch.randint(2,5,(batch_size,))
tgt_len = torch.randint(2,5,(batch_size,))
# 生成两个序列，每个序列都有一个随机生成的长度，最小为2，最大为5
# src_seq
# tgt_seq
print(src_len)
print(tgt_len)

tensor([4, 3])
tensor([4, 2])


In [4]:
# 序列单词总数，单词表大小
max_num_src_words = 8
max_num_tgt_words = 8
#  序列最大程度
max_src_seq_len = 5
max_tgt_seq_len = 5

src_len = torch.Tensor([2,4]).to(torch.int32)
tgt_len = torch.Tensor([4,3]).to(torch.int32)

# 以单词索引构成的句子
# src_seq = torch.cat([torch.unsequeeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)),0) \ 
#                     for L in src_len ])
src_seq = [ F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)) for L in src_len ]

print(src_seq)
# 71 1364
tgt_seq = [ F.pad(torch.randint(1,max_num_tgt_words,(L,)),(0,max_src_seq_len-L)) for L in tgt_len ]
print(tgt_seq)

[tensor([3, 2, 0, 0, 0]), tensor([2, 1, 3, 4, 0])]
[tensor([2, 2, 5, 3, 0]), tensor([3, 7, 4, 0, 0])]


In [5]:
# 对每个张量做pad,拼在一起
# torch.unsequeeze升维度
# src_seq = [torch.unsqueeze(T,dim=1) for T in src_seq]
# print(src_seq)
# torch.cat两个张量拼接，只能在已有的维度上，横向拼或者纵向拼
# 默认值为0
src_seq = torch.stack(tuple(src_seq),0)
print(src_seq)
tgt_seq = torch.stack(tuple(tgt_seq),0)
print(tgt_seq)

tensor([[3, 2, 0, 0, 0],
        [2, 1, 3, 4, 0]])
tensor([[2, 2, 5, 3, 0],
        [3, 7, 4, 0, 0]])


In [6]:
# 构造embedding
model_dim = 4

src_embedding_table = nn.Embedding(max_num_src_words+1,model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1,model_dim)
print(src_embedding_table.weight)

Parameter containing:
tensor([[ 0.6906,  1.2310,  0.3961, -0.5545],
        [-0.2322,  1.2972,  0.3335, -0.2356],
        [-0.9117, -0.0281,  0.7198,  0.0489],
        [ 1.8042,  0.3087,  0.0875,  0.7060],
        [ 0.2757, -0.8721, -0.2537, -0.2445],
        [ 0.2626, -0.3904, -0.4275, -1.5268],
        [-0.0972,  0.0288, -0.6968,  0.3374],
        [-0.1964,  2.2362, -0.0749, -1.5696],
        [-1.0461,  0.2858,  0.5081, -1.0913]], requires_grad=True)


In [7]:
src_embedding = src_embedding_table(src_seq)
print(src_seq)
print(src_embedding)
# 迪奥用embedding的forward方法
tgt_embedding = tgt_embedding_table(tgt_seq)

tensor([[3, 2, 0, 0, 0],
        [2, 1, 3, 4, 0]])
tensor([[[ 1.8042,  0.3087,  0.0875,  0.7060],
         [-0.9117, -0.0281,  0.7198,  0.0489],
         [ 0.6906,  1.2310,  0.3961, -0.5545],
         [ 0.6906,  1.2310,  0.3961, -0.5545],
         [ 0.6906,  1.2310,  0.3961, -0.5545]],

        [[-0.9117, -0.0281,  0.7198,  0.0489],
         [-0.2322,  1.2972,  0.3335, -0.2356],
         [ 1.8042,  0.3087,  0.0875,  0.7060],
         [ 0.2757, -0.8721, -0.2537, -0.2445],
         [ 0.6906,  1.2310,  0.3961, -0.5545]]], grad_fn=<EmbeddingBackward>)


### 2. position embedding

position embedding公式
$$
\begin{aligned}
P E_{(p o s, 2 i)} &=\sin \left(p o s / 10000^{2 i / d_{\mathrm{model}}}\right) \\
P E_{(\text {pos }, 2 i+1)} &=\cos \left(p o s / 10000^{2 i / d_{\mathrm{model}}}\right)
\end{aligned}
$$

In [45]:
# 构造位置编码
# sin/cos 泛化能力比较强，具有对称性和唯一性
# 
max_position_len = 5

# 奇数列用cos，偶数列用sin
pos_mat = torch.arange(max_position_len)
print(pos_mat)
# pos决定行，i决定列
pos_mat = pos_mat.reshape((-1,1))
print(pos_mat)

import numpy as np
# 列矩阵
i_mat = torch.Tensor(np.arange(0,model_dim,2))/model_dim
print(i_mat)
i_mat = i_mat.unsqueeze(0)
print(i_mat)

i_mat = torch.pow(10000, i_mat)
print(i_mat)
pos_mat = pos_mat.float()
i_mat = i_mat.float()

tensor([0, 1, 2, 3, 4])
tensor([[0],
        [1],
        [2],
        [3],
        [4]])
tensor([0.0000, 0.5000])
tensor([[0.0000, 0.5000]])
tensor([[  1., 100.]])


In [46]:
pe_embedding_table = torch.zeros(max_position_len, model_dim).float()
print(pe_embedding_table)
pe_embedding_table[:,0::2] = torch.sin(pos_mat/i_mat)
print(pe_embedding_table)
pe_embedding_table[:,1::2] = torch.cos(pos_mat/i_mat)
print(pe_embedding_table)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0000,  0.0100,  0.0000],
        [ 0.9093,  0.0000,  0.0200,  0.0000],
        [ 0.1411,  0.0000,  0.0300,  0.0000],
        [-0.7568,  0.0000,  0.0400,  0.0000]])
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992]])


In [36]:
print(torch.cuda.is_available())

True


In [56]:
pe_embedding = nn.Embedding(max_position_len,model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table,requires_grad=False)
print(pe_embedding.weight)

src_pos = [torch.arange(max(src_len)) for _ in src_len]
print(src_pos)

src_pe_embedding = pe_embedding(src_pos)
# tgt_pe_embedding = pe_embedding(tgt_pos)
print(src_pe_embedding)

Parameter containing:
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992]])
[tensor([0, 1, 2, 3]), tensor([0, 1, 2, 3])]


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list